## H5N1 in Genspectrum

#### Goal: Use as many H5N1 sequences from NCBI as possible

Current steps:

You can recreate the findings of this notebook by using the `loculus-ingest` pipeline and running `snakemake process_alignments`.

1. Download H5N1 datasets using the NCBI CLI tool. Sadly, there used to exist a H5N1 taxon, but this is no longer used. The best approach is to now use NCBI virus to filter Influenza A based on genotype, however the CLI does not have this option. (They are working on genotype/serotype based filtering), until then I filter Influenza A based on if the sequences contain H5N1 in their title. (See https://github.com/ncbi/datasets/issues/407 for details, they suggest using taxonId 2955291, but this has no impact on the final results). We download all sequences from taxonId 197911 = 1'129'000 sequences total (as of October 14, 2024), for comparison taxonId 2955291 = 1'128'989.

```
datasets download virus genome taxon 197911 --no-progressbar --filename results/ncbi_dataset.zip 
unzip -jp results/ncbi_dataset.zip ncbi_dataset/data/genomic.fna | seqkit seq -w0  > results/sequences.fasta
```

2. Filter out sequences which do not have H5N1 in their title: "results/sequences_filtered.fasta"

I also download the accessions of all H5N1 genotypes shown on NCBI virus from the webpage: "H5N1_accession_list.tsv". For comparison, alternatively I could use the datasets CLI to download only the sequences in this list, but this would require continuous updating.


In [19]:
from Bio import SeqIO

file_path = "H5N1_accession_list.tsv"

with open(file_path, "r") as file:
    file.readline()
    ncbi_virus_accessions = {line.split('\t')[0] for line in file}

print(f"NCBI Virus finds {len(ncbi_virus_accessions)} H5N1 accessions")

input_seq = "results/sequences_filtered.fasta"

with open(input_seq) as f:
    records = SeqIO.parse(f, "fasta")
    fasta_accessions = {record.id.split(".")[0] for record in records}

print(f"We find {len(fasta_accessions)} accessions using filtering.")

print(f"We find {len(fasta_accessions - ncbi_virus_accessions)} more accessions.")

NCBI Virus finds 63429 H5N1 accessions
We find 64799 accessions using filtering.
We find 1455 more accessions.


In [20]:
missing = (ncbi_virus_accessions - fasta_accessions)
print(f"We are missing {len(missing)} accessions.")

We are missing 85 accessions.


In [22]:
print("Missing accessions:")
with open("results/sequences.fasta") as f:
    records = SeqIO.parse(f, "fasta")
    for record in records:
        if record.id.split(".")[0] in missing:
            print(record.description)

Missing accessions:
LC106080.1 Influenza A virus NP gene for nucleocapsid protein, complete cds
LC106090.1 Influenza A virus NA gene for neuraminidase, complete cds
LC106103.1 Influenza A virus NEP, NS1 genes for nonstructural protein 2, nonstructural protein 1, complete cds
LC106056.1 Influenza A virus PB1 gene for polymerase PB1, complete cds
LC106063.1 Influenza A virus PA gene for polymerase PA, complete cds
LC106078.1 Influenza A virus NP gene for nucleocapsid protein, complete cds
LC106062.1 Influenza A virus PA gene for polymerase PA, complete cds
LC106064.1 Influenza A virus PA gene for polymerase PA, complete cds
LC106106.1 Influenza A virus NEP, NS1 genes for nonstructural protein 2, nonstructural protein 1, complete cds
LC106055.1 Influenza A virus PB1 gene for polymerase PB1, complete cds
LC106109.1 Influenza A virus NEP, NS1 genes for nonstructural protein 2, nonstructural protein 1, complete cds
LC106061.1 Influenza A virus PA gene for polymerase PA, complete cds
LC106065

3. Align each segment using the nextclade dataset: 
```
for i in {1..8}; do
nextclade run results/sequences_filtered.fasta --output-tsv nextclade_seg$i.tsv \
    --server https://raw.githubusercontent.com/genspectrum/nextclade-datasets/reduceMinSeedCover/data --dataset-name flu/h5n1/seg$i
done
```

Using https://github.com/nextstrain/nextclade_data/pull/217 with all default alignment values we fail to align 6349 of the filtered sequences (equivalent to 6281 of the H5N1 sequences downloaded from NCBI virus).

With https://github.com/GenSpectrum/nextclade-datasets/pull/2 I improve this leading to:

In [23]:
with open("results/nextclade_merged.tsv", "r") as file:
    file.readline()
    aligned_sequences = {line.split('\t')[0].split(".")[0] for line in file}

failed_alignment = (fasta_accessions - aligned_sequences)

print(f"We are able to align {len(aligned_sequences)} accessions.")
print(f"This mean a total of {len(fasta_accessions - aligned_sequences)} sequences failed to align.")

We are able to align 64443 accessions.
This mean a total of 356 sequences failed to align.


In [24]:
print("Sequences that failed to align:")
count = 0
with open("unaligned_sequences.fasta", "w", encoding="utf-8") as output_file:
    with open("results/sequences.fasta") as f:
        records = SeqIO.parse(f, "fasta")
        for record in records:
            if record.id.split(".")[0] in failed_alignment:
                print(record.description)
                if 'segment 8' in record.description:
                    count += 1
                    output_file.write(f">{record.description}\n{record.seq}\n")
print(f"Found {count} segment 8 sequences")

Sequences that failed to align:
OP269992.1 Influenza A virus (A/Vulpes vulpes/USA/103994/2022(H5N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds
OQ584713.1 Influenza A virus (A/black vulture/Georgia/W22-749A/2022(H5N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds
PP782343.1 Influenza A virus (A/Sterna hirundo/Cabo FrioBR/0613-N/2024(H5N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds
PP853092.1 Influenza A virus (A/chicken/Serbia/22337-21/2021(H5N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds
CY099014.1 Influenza A virus (A/environment/Xinjiang/6/2009(H5N1)) nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds
OR421138.1 Influenza A virus (A/green-winged teal/Minnesota/UGAI22-3856/2022(H5N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1)